In [4]:
!pip install nsetools pandas nsepy PyPortfolioOpt bokeh

  Using cached nsetools-1.0.11-py3-none-any.whl (9.6 kB)
  Using cached nsepy-0.8-py3-none-any.whl
  Using cached PyPortfolioOpt-1.4.1-py3-none-any.whl (56 kB)
  Using cached lxml-4.6.3-cp38-cp38-manylinux2014_x86_64.whl (6.8 MB)
  Using cached dateutils-0.6.12-py2.py3-none-any.whl (5.7 kB)
  Using cached cvxpy-1.1.12.tar.gz (1.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached osqp-0.6.2.post0-cp38-cp38-manylinux2014_x86_64.whl (215 kB)
  Using cached ecos-2.0.7.post1.tar.gz (126 kB)
  Using cached scs-2.1.3.tar.gz (147 kB)
  Using cached qdldl-0.1.5.post0-cp38-cp38-manylinux2014_x86_64.whl (921 kB)
  Created wheel for cvxpy: filename=cvxpy-1.1.12-cp38-cp38-linux_x86_64.whl size=3724523 sha256=abdf5a0d47dab4c450fa21c2a7575dc9d3cdf190a797899f22d02fae2a4cc881
  Stored in directory: /home/jovyan/.cache/pip/wheels/df/06/53/181f6845804251b9a5d843803577d2772f1aea96cb10b51e5f
  Created wheel for e

In [ ]:
from fetcher import Nifty500Fetcher

nff = Nifty500Fetcher()
nff.read_list(update=True)

In [ ]:
nff.fetch_data()

In [ ]:
nff.fetch_metadata()

In [ ]:
from cleaner import Nifty500Cleaner

nfc = Nifty500Cleaner()
nfc.clean_metadata()
nfc.clean_ohlc_data()

In [ ]:
from processor import StockProcessor

sp = StockProcessor('3MINDIA', meta_data_loc='data/cleaned/Metadata/nifty500.json')

In [ ]:
sp.process_metrics()

In [ ]:
sp.stock.metadata

In [ ]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", 
                    "data/cleaned/Metadata/nifty500.json", 
                    "data/cleaned/Metadata/proc_nifty500.json")

In [ ]:
pp.process_metrics()

In [ ]:
import pandas as pd
pd.read_csv('data/cleaned/OHLC/Nifty500/M&M.csv')

In [ ]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", "data/cleaned/Metadata/nifty500.json")
pp.process_close(30)

In [ ]:
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer

opt1 = EffOptimizer(pp)
# opt2 = HRPOptimizer(pp)
# opt3 = CLAOptimizer(pp)

In [ ]:
opt1.optimize_max_sharpe()

In [ ]:
opt1.portfolio.statistics

In [ ]:
opt1.optimize_min_volatility()

In [ ]:
opt1.portfolio.statistics

In [ ]:
opt2 = HRPOptimizer(pp)
opt2.optimize()

In [ ]:
opt2.portfolio.statistics

In [ ]:
temp_df = (pp.close_matrix.pct_change(30).tail(1)*100 > 0)
temp_df

In [ ]:
opt3 = CLAOptimizer(pp)
opt3.optimize_max_sharpe()

In [ ]:
opt3.portfolio.statistics

In [ ]:
opt3.optimize_min_volatility()
opt3.portfolio.statistics

In [ ]:
import pandas as pd
df = pd.read_csv('data/raw/nifty_500_metadata.csv')#.to_dict(orient='records')

In [ ]:
some = df[df['symbol']=='M&amp;M'].to_dict(orient='records')

In [ ]:
some

In [ ]:
from fetcher import Nifty500Fetcher

nff = Nifty500Fetcher()
nff.fetch_index('NIFTY 500')

In [1]:
from simulate import Backtesting, Agent
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer

agents = [Agent("CLA_MAX_SHARPE", 'navy', CLAOptimizer(), 1000000, optimizer_type='max_sharpe',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80),
         Agent("EFF_MIN_VOL", 'green', EffOptimizer(), 1000000, optimizer_type='min_vol',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80),
         Agent("EFF_MAX_SHARPE", 'purple', EffOptimizer(), 1000000, optimizer_type='max_sharpe',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80),
         Agent("HRP", 'orange', HRPOptimizer(), 1000000, metadata_loc="data/processed/Metadata/nifty500.json", 
                           single_day_cash=0.80),
         Agent("CLA_MIN_VOL", 'brown', CLAOptimizer(), 1000000, optimizer_type='min_vol',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80)]

benchmarks = [
    {
        'name':'Nifty500',
        'data_loc':'data/raw/Nifty500/Index/NIFTY500.csv',
        'color':'red'
    }
]

bt = Backtesting(proc_ohlc_loc="data/processed/OHLC/Nifty500", 
              proc_metadata_loc="data/processed/Metadata/nifty500.json", bband_margins=False,
                 lookback_period=1,rebalance_period=1,
                 bband_ma=20, benchmarks=benchmarks, #process_metrics=True,
                agents=agents)

In [2]:
bt.cret_plot()

Loading BokehJS ...

In [3]:
bt.portfolio_plot()

In [ ]:
bt.backtest()

Day: 0

{
    "Portfolio Name": "CLA_MAX_SHARPE",
    "Date": "2011-06-01",
    "Single Day Cash Allocation": 80.0,
    "Initial Portfolio Value": 1000000,
    "Current Portfolio Value": 1000000,
    "Cumulative Portfolio Return": 0.0,
    "Amount Invested": 0,
    "Amount Reserved": 1000000,
    "Reallocation Day": false
}

Day: 1


/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:8525: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/opt/conda/lib/python3.8/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/opt/conda/lib/python3.8/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/opt/conda/lib/python3.8/site-packages/sklearn/covariance/_shrunk_covariance.py:203: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn("Only one sample available. "
/opt/conda/lib/python3.8/site-packages/sklearn/covariance/_empirical_covariance.py:88: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn("Only one sample available. "


Portfolio Optimization Error: Singular matrix

PORTFOLIO: {}

Optimizer PORTFOLIO: {}

ORDERS: {}


{
    "Portfolio Name": "CLA_MAX_SHARPE",
    "Date": "2011-06-02",
    "Single Day Cash Allocation": 80.0,
    "Initial Portfolio Value": 1000000,
    "Current Portfolio Value": 1000000.0,
    "Cumulative Portfolio Return": 0.0,
    "Amount Invested": 0,
    "Amount Reserved": 1000000.0,
    "Reallocation Day": true
}

Day: 2


/opt/conda/lib/python3.8/site-packages/pypfopt/risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/pypfopt/risk_models.py:87: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


Portfolio Optimization Error: '>' not supported between instances of 'NoneType' and 'float'

PORTFOLIO: {}

Optimizer PORTFOLIO: {}

ORDERS: {}


{
    "Portfolio Name": "CLA_MAX_SHARPE",
    "Date": "2011-06-03",
    "Single Day Cash Allocation": 80.0,
    "Initial Portfolio Value": 1000000,
    "Current Portfolio Value": 1000000.0,
    "Cumulative Portfolio Return": 0.0,
    "Amount Invested": 0,
    "Amount Reserved": 1000000.0,
    "Reallocation Day": true
}

Day: 3

PORTFOLIO: {}

Optimizer PORTFOLIO: {'AKZOINDIA': 126, 'ECLERX': 101, 'HONAUT': 14, 'VIPIND': 40, 'CONCOR': 26, 'THERMAX': 44, 'SKFINDIA': 41, 'PGHH': 13, 'COLPAL': 25, 'HDFC': 33, 'DRREDDY': 13, 'OFSS': 10, 'GILLETTE': 10, 'TECHM': 26, 'LT': 9, 'NESCO': 27, 'BEL': 9, 'GODREJPROP': 18, 'CRISIL': 2, 'TCS': 10, 'TATAMOTORS': 11, 'NESTLEIND': 3, 'WABCOINDIA': 11, 'DIVISLAB': 13, 'TATAMTRDVR': 16, 'ACC': 9, 'TITAN': 2, 'KANSAINER': 10, 'RELIANCE': 9, 'ICICIBANK': 7, 'ABBOTINDIA': 5, 'HDFCBANK': 3, 'EICHERMOT': 5, 'BOSCHLTD

...


PORTFOLIO: {'TECHM': 59, 'HCLTECH': 58, 'ICRA': 40, 'ECLERX': 29, 'JUSTDIAL': 18, 'PAGEIND': 8, 'LAXMIMACH': 16, 'SKFINDIA': 35, 'SCHAEFFLER': 14, 'SYMPHONY': 120, 'EICHERMOT': 12, 'HEROMOTOCO': 8, 'SUNDARMFIN': 29, 'MARUTI': 5, 'NESCO': 16, 'PERSISTENT': 57, 'JSWSTEEL': 19, 'VSTIND': 13, 'HONAUT': 13, 'PGHH': 7, 'THERMAX': 42, 'WABCOINDIA': 11, 'BAJAJHLDNG': 5, 'CRISIL': 10, 'ABBOTINDIA': 24, 'CARERATING': 7, 'BEL': 13, 'CADILAHC': 22, 'HAVELLS': 8, 'AXISBANK': 37, 'TCS': 2, 'BLUEDART': 6, 'SOLARINDS': 39, 'SHREECEM': 2, 'BRITANNIA': 6, 'BASF': 40, 'BOSCHLTD': 1, '3MINDIA': 2, 'DIVISLAB': 2, 'SHRIRAMCIT': 2}

Optimizer PORTFOLIO: {'SYMPHONY': 117, 'TECHM': 58, 'EICHERMOT': 12, 'ICRA': 39, 'HCLTECH': 56, 'PERSISTENT': 62, 'SOLARINDS': 38, 'ABBOTINDIA': 26, 'HONAUT': 12, 'LAXMIMACH': 16, 'PAGEIND': 8, 'WABCOINDIA': 13, 'SCHAEFFLER': 14, 'ECLERX': 30, 'BASF': 31, 'PGHH': 7, 'SKFINDIA': 32, 'THERMAX': 38, 'JUSTDIAL': 18, 'BLUEDART': 7, 'CADILAHC': 20, 'BEL': 14, 'SUNDARMFIN': 26, 'NESCO'

Day: 809

PORTFOLIO: {'TECHM': 55, 'HCLTECH': 56, 'ICRA': 38, 'ECLERX': 33, 'JUSTDIAL': 19, 'PAGEIND': 7, 'LAXMIMACH': 16, 'SKFINDIA': 40, 'SCHAEFFLER': 14, 'SYMPHONY': 117, 'EICHERMOT': 11, 'HEROMOTOCO': 9, 'SUNDARMFIN': 26, 'MARUTI': 7, 'NESCO': 17, 'PERSISTENT': 59, 'JSWSTEEL': 15, 'VSTIND': 12, 'HONAUT': 12, 'PGHH': 7, 'THERMAX': 36, 'WABCOINDIA': 12, 'BAJAJHLDNG': 6, 'CRISIL': 12, 'ABBOTINDIA': 25, 'CARERATING': 4, 'BEL': 20, 'CADILAHC': 22, 'AXISBANK': 37, 'TCS': 5, 'BLUEDART': 8, 'SOLARINDS': 38, 'SHREECEM': 2, 'BRITANNIA': 7, 'BASF': 26, 'BOSCHLTD': 1, '3MINDIA': 2, 'DIVISLAB': 4, 'SHRIRAMCIT': 2}

Optimizer PORTFOLIO: {'SYMPHONY': 119, 'TECHM': 58, 'EICHERMOT': 11, 'ICRA': 40, 'HCLTECH': 55, 'SOLARINDS': 40, 'PERSISTENT': 55, 'LAXMIMACH': 17, 'ABBOTINDIA': 24, 'HONAUT': 12, 'PAGEIND': 7, 'ECLERX': 35, 'WABCOINDIA': 13, 'BASF': 34, 'BEL': 19, 'SKFINDIA': 34, 'BLUEDART': 8, 'PGHH': 7, 'SCHAEFFLER': 12, 'THERMAX': 36, 'SUNDARMFIN': 31, 'JUSTDIAL': 17, 'HEROMOTOCO': 9, 'CRISIL': 1

/opt/conda/lib/python3.8/site-packages/pypfopt/discrete_allocation.py:210: RuntimeWarning: invalid value encountered in true_divide
  current_weights /= current_weights.sum()



PORTFOLIO: {'TECHM': 56, 'HCLTECH': 54, 'ICRA': 39, 'ECLERX': 34, 'JUSTDIAL': 15, 'PAGEIND': 8, 'LAXMIMACH': 17, 'SKFINDIA': 32, 'SCHAEFFLER': 12, 'SYMPHONY': 115, 'EICHERMOT': 11, 'HEROMOTOCO': 10, 'SUNDARMFIN': 33, 'MARUTI': 8, 'NESCO': 13, 'PERSISTENT': 56, 'JSWSTEEL': 18, 'VSTIND': 12, 'HONAUT': 11, 'PGHH': 6, 'THERMAX': 34, 'WABCOINDIA': 12, 'BAJAJHLDNG': 3, 'CRISIL': 12, 'ABBOTINDIA': 24, 'CARERATING': 4, 'BEL': 19, 'CADILAHC': 18, 'AXISBANK': 25, 'TCS': 4, 'BLUEDART': 8, 'SOLARINDS': 39, 'SHREECEM': 2, 'BRITANNIA': 6, 'BASF': 34, 'BOSCHLTD': 1, '3MINDIA': 2, 'DIVISLAB': 4, 'SHRIRAMCIT': 4, 'BAJAJFINSV': 3, 'ABB': 1}

Optimizer PORTFOLIO: {'SYMPHONY': 120, 'TECHM': 57, 'EICHERMOT': 11, 'ICRA': 38, 'HCLTECH': 55, 'SOLARINDS': 39, 'LAXMIMACH': 17, 'PERSISTENT': 55, 'ABBOTINDIA': 24, 'HONAUT': 12, 'PAGEIND': 8, 'ECLERX': 33, 'WABCOINDIA': 12, 'BASF': 34, 'BEL': 18, 'SKFINDIA': 31, 'BLUEDART': 8, 'SCHAEFFLER': 12, 'SUNDARMFIN': 34, 'PGHH': 6, 'JUSTDIAL': 18, 'THERMAX': 34, 'HEROMOTO

Day: 814
